In [8]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Will plot in separate pop-up window
%matplotlib qt

In [9]:
# Read in GISTEMP data as dataframe
gistemp_file = 'gistemp_monthly.csv'
df_full = pd.read_csv(gistemp_file, header=1)
df_full = df_full.set_index('Year')

# Drop non-monthly columns
df = df_full.iloc[:, :12]

# Drop 2023 (incomplete)
df = df.head(len(df) - 1)

# Convert to floats
df = df.astype(float)

# Create column for average of all values
df['Avg'] = df.mean(axis=1)
df

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Avg
Year,,,,,,,,,,,,,
1880,-0.19,-0.25,-0.09,-0.17,-0.10,-0.21,-0.18,-0.11,-0.15,-0.24,-0.22,-0.18,-0.174167
1881,-0.20,-0.15,0.03,0.05,0.05,-0.19,0.00,-0.04,-0.16,-0.22,-0.19,-0.08,-0.091667
1882,0.16,0.13,0.04,-0.16,-0.14,-0.22,-0.17,-0.08,-0.15,-0.24,-0.17,-0.36,-0.113333
1883,-0.30,-0.37,-0.13,-0.19,-0.18,-0.08,-0.08,-0.14,-0.23,-0.12,-0.24,-0.11,-0.180833
1884,-0.13,-0.09,-0.37,-0.40,-0.34,-0.35,-0.31,-0.28,-0.28,-0.25,-0.34,-0.31,-0.287500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,0.82,0.85,0.88,0.89,0.83,0.77,0.82,0.77,0.80,1.02,0.83,0.92,0.850000
2019,0.93,0.95,1.17,1.02,0.85,0.91,0.94,0.95,0.93,1.01,1.00,1.09,0.979167
2020,1.18,1.25,1.17,1.13,1.02,0.92,0.90,0.88,0.99,0.89,1.11,0.81,1.020833


In [11]:
def monthly_timeseries(column='Avg'):
    
    # Initialize lists/counter, set values for colorbar and number of poitns
    temp_list = []
    year_list = []
    counter = 0
    num_points = len(df) - 1
    max_color = 1.25
    min_color = -max_color

    while True:

        # Gather values cumulatively
        avg_temp = df[column].iloc[counter]
        year_list.append(int(df.index[counter]))
        temp_list.append(avg_temp)

        # Line plot
        plt.rcParams["figure.figsize"] = (10, 10)
        plt.plot(year_list, temp_list, color='black')
        
        # Horizontal line for baseline temperature
        plt.axhline(y=0, color='black', linestyle='dashed')

        # Scatter plot
        plt.scatter(year_list, temp_list, c=temp_list, cmap='seismic')
        plt.clim(min_color, max_color)

        # Formatting
        plt.xlabel('Year')
        plt.ylabel('Temperature Anomaly (Degrees Celcius)')
        ymin = min(df[column]) * 1.1
        ymax = max(df[column]) * 1.1
        plt.ylim(ymin, ymax)
        title = 'GISTEMP Temperature Progression'
        if column != 'Avg':
            title += ' ('  + column +')'                    
        plt.title(title)
        plt.draw()
        plt.grid()

        # Could ideally get the colorbar to match up with the y-axis but that's a bit tricky
        plt.colorbar(ticks=np.linspace(min_color, max_color, 11)).set_label('Degrees Celcius')
        plt.pause(0.05)

        if counter < num_points:
            plt.clf()
        else:
            break

        counter += 1

In [12]:
# Similar to the above plot, but including every monthly data point
def overall_timeseries():

    # Get a list of all monthly values
    values = []
    dates = []

    # Create list of strings for each month
    month_list = []
    for i in range(12):
        month_num = str(i+1)
        if len(month_num) < 2:
            month_num = '0' + month_num
        month_list.append(month_num)

    # Collect datetime and values for each entry in dataframe
    for i in range(len(df)):
        year = df.iloc[i].name
        months = df.iloc[i].index
        val_i = df.iloc[i].values
        month_counter = 0
        for j in range(len(val_i)-1):
            values.append(val_i[j])
            date_str = str(year) + '-' + month_list[month_counter]
            dt = datetime.datetime.strptime(date_str, '%Y-%m')
            dates.append(dt)
            month_counter += 1

    # Initialize lists/counter, set values for colorbar and number of poitns
    temp_list = []
    year_list = []
    counter = 0
    num_points = len(values) - 1
    max_color = 1.25
    min_color = -max_color

    while True:

        # Gather values cumulatively
        temp_list.append(values[counter])
        year_list.append(dates[counter])

        # Line plot
        plt.rcParams["figure.figsize"] = (10, 10)
        plt.plot(year_list, temp_list, color='black')

        # Horizontal line for baseline temperature
        plt.axhline(y=0, color='black', linestyle='dashed')

        # Scatter plot
        plt.scatter(year_list, temp_list, c=temp_list, cmap='seismic')
        plt.clim(min_color, max_color)

        # Formatting
        plt.xlabel('Year')
        plt.ylabel('Temperature Anomaly (Degrees Celcius)')
        ymin = min(values) * 1.1
        ymax = max(values) * 1.1
        plt.ylim(ymin, ymax)
        title = 'GISTEMP Temperature Progression'
        plt.title(title)
        plt.draw()
        plt.grid()

        # Could ideally get the colorbar to match up with the y-axis but that's a bit tricky
        plt.colorbar(ticks=np.linspace(min_color, max_color, 11)).set_label('Degrees Celcius')
        plt.pause(0.0000001)

        if counter < num_points:
            plt.clf()
        else:
            break

        counter += 1

In [13]:
monthly_timeseries('Jan')

In [6]:
overall_timeseries()